# Tutorial

Nangs is a python module built on top of pytorch to solve partial differential equations (PDEs).

## How does it work ?

Let's assume we want to solve the following PDE:

\begin{equation}
    \frac{\partial \phi}{\partial t} + u \frac{\partial \phi}{\partial x} = 0
\end{equation}

Different numerical techniques that solve this problem exist, and all of them are based on finding an approximate function that satisfies the PDE. Traditional numerical methods discretize the domain into small elements where a form of the solutions is assumed (for example, a constant) and then the final solution is composed as a piece-wise, discontinuous function.

Nangs uses the property of neural networks (NNs) as universal function approximators to find a continuous and derivable solution to the PDE, that requires significant less computing resources compared with traditional techniques and with the advantage of including the free-parameters as part of the solution.

The independen variables (i.e, $x$ and $t$) are used as input values for the NN, and the solution (i.e. $\phi$) is the output. In order to find the solution, at each step the NN outputs are derived w.r.t the inputs. Then, a loss function that matches the PDE is built and the weights are updated accordingly. If the loss function goes to zero, we can assume that our NN is indeed the solution to our PDE.

We will see how to do that in this basic tutorial.

## Quick start

Here you can find the code required to solve the above mentioned PDE.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import math
import numpy as np 
import matplotlib.pyplot as plt 

import torch
cuda = False
device = torch.device("cuda:0" if torch.cuda.is_available() and cuda else "cpu")
print(device)

# import nangs
from nangs import PDE
from nangs.bocos import PeriodicBoco, DirichletBoco

# define custom PDE
class MyPDE(PDE):
    def __init__(self, inputs=None, outputs=None, params=None):
        super().__init__(inputs, outputs, params)
    def computePdeLoss(self, grads, params): 
        # here is where the magic happens
        dpdt, dpdx = grads['p']['t'], grads['p']['x']
        u = params['u']
        return dpdt + u*dpdx

# instanciate pde
pde = MyPDE(inputs=['x', 't'], outputs=['p'], params=['u'])

# define input values
x = np.linspace(0,1,10)
t = np.linspace(0,1,10)
u = np.array([1])

pde.setValues({'x': x, 't': t, 'u': u})

# periodic b.c for the space dimension
x1, x2 = np.array([0]), np.array([1])
boco = PeriodicBoco(pde.inputs, {'x': x1, 't': t}, {'x': x2, 't': t})
pde.addBoco(boco)

# initial condition (dirichlet for temporal dimension)
p0 = np.sin(2.*math.pi*x)
boco = DirichletBoco(pde.inputs, pde.outputs, {'x': x, 't': np.array([0])}, {'p': p0})
pde.addBoco(boco)

# define solution topology
topo = {'layers': 5, 'neurons': 32, 'activations': 'relu'}
pde.buildModel(topo, device)

# set optimization parameters
pde.setSolverParams(lr=0.01,epochs=1000, batch_size=100)

# find the solution
pde.solve(device, each_epoch_print=100) 

# evaluate the solution
x = np.linspace(0,1,50)
t = np.linspace(0,1,4)
for _t in t:
    p0 = np.sin(2.*math.pi*(x-u*_t))
    p = pde.eval({'x': x, 't': np.array([_t])})
    plt.plot(x, p, 'k')
    plt.plot(x, p0)
    plt.show()

cpu


ValueError: not enough values to unpack (expected 2, got 1)

## Step by step guide

Let's go through the code step by step. First, we import the nangs module to acces its predefined classes and operations to solve PDEs with NNs.

In [ ]:
from nangs import PDE

%load_ext autoreload
%autoreload 2

### Defining the PDE

Then, we have to instanciate a PDE in order to solve it. We can use a predefined PDE or build our custom one. In this tutorial we are going to solve the one-dimensional advection equation with a custom implementation.

In [ ]:
class MyPDE(PDE):
    def __init__(self, inputs=None, outputs=None, params=None):
        super().__init__(inputs, outputs, params)
    def computePdeLoss(self, grads, params): 
        # here is where the magic happens
        dpdt, dpdx = grads['p']['t'], grads['p']['x']
        u = params['u']
        return dpdt + u*dpdx

When defining a custom PDE we are required to overwrite the **computePdeLoss** method. This is the function that will be called during the optimization process, and we must ensure that the value returned matches our PDE. If not, we are going to solve a wrong equation.

We build the loss function by combining the derivatives of the NN ouputs w.r.t the inputs and free-parameters in the correct way. Notice that we use keys to retrieve the values ('p' for the output, 'x' and 't' for the inputs and 'u' for the parameter). We must ensure that this keys are created accordingly when the PDE is instanciated.

In [ ]:
pde = MyPDE(inputs=['x', 't'], outputs=['p'], params=['x'])
pde.summary()

We have created two inputs 'x' and 't', an output 'p' and a parameter 'u'. The keys for the different variables must be unique and passed as a list. The order of each key in the list determines the position of the input/output in the corresponding NN layer. 

Note that a free-parameter can also be included as input to the NN.

In [ ]:
# short version
pde = MyPDE(inputs=['t','x','u'], outputs=['p'], params=['u'])
pde.summary()

Different errors may arise when trying to define several inputs/outputs/parameters with the same key. The only allowed repeated values are input-param pairs.

In [ ]:
#pde = MyPDE(inputs=['x','x'], outputs=['p'], params=['u'])
#pde = MyPDE(inputs=['t','x'], outputs=['x'], params=['u'])
#pde = MyPDE(inputs=['t','x'], outputs=['p', 'p'], params=['u'])
#pde = MyPDE(inputs=['t','x'], outputs=['p'], params=['p'])
pde = MyPDE(inputs=['t','x'], outputs=['p'], params=['u'])
pde.summary()

At this point we only have declared keys to our variables. In order to solve the PDE we have to specify actual values for all this variables that will be evaluated during the optimization process. We can define this values with the corresponding functions. Values are passed as dictionaries, with the key of the variable and the values as a numpy array of ine dimension. Values for outputs cannot be set.

In [ ]:
import numpy as np 

pde = MyPDE(inputs=['x', 't', 'u'], outputs=['p'], params=['u'])

# set input values
x = np.array([0, 0.25, 0.5, 0.75, 1.0])
t = np.array([0,0.25, 0.5, 0.75, 1.0])
u = np.array([1])

pde.setValues({'x': x, 't': t, 'u': u})

pde.summary()

### Defining boundary conditions

We could attempt to start finding the solution at this point, and probably we would find one. Nevertheless that would be a trivial solution since a PDE is constrained by boundary conditions. We can add them as follow.

In [ ]:
import math
from nangs.bocos import PeriodicBoco, DirichletBoco

pde = MyPDE(inputs=['x', 't'], outputs=['p'], params=['u'])

x = np.array([0, 0.25, 0.5, 0.75, 1.0])
t = np.array([0,0.25, 0.5, 0.75, 1.0])
u = np.array([1])

pde.setValues({'x': x, 't': t, 'u': u})

# periodic b.c for the space dimension
x1, x2 = np.array([0]), np.array([1])
boco = PeriodicBoco(pde.inputs, {'x': x1, 't': t}, {'x': x2, 't': t})
pde.addBoco(boco)

# initial condition (dirichlet for temporal dimension)
p0 = np.sin(2.*math.pi*x)
boco = DirichletBoco(pde.inputs, pde.outputs, {'x': x, 't': np.array([0])}, {'p': p0})
pde.addBoco(boco)

pde.summary()
pde.bocoSummary()

Boundary conditions must be specified one at a time. Depending on the **type** a different number of parameters are required. 

A PERIODIC boundary condition requires pairs of values that will be enforced to be equal at every step using a mean squared optimization.

A DIRICHLET boundary condition requires a specific value for the outputs that will be matched using a typical mean squared optimization.

### NN topology and optimization parameters

We encode the solution to the PDE using a multi-layer perceptron (MLP).

There are two ways to define the topolgy.

In [ ]:
# compact form
topo = {'layers': 3, 'neurons': 32, 'activations': 'relu'}

# TODO: layer by layer
#topo = [{'layers': 1, 'neurons': 256, 'activations': 'relu'},
#        {'layers': 2, 'neurons': 512, 'activations': 'relu'},
#        {'layers': 1, 'neurons': 32, 'activations': 'sigmoid'}]


pde.buildModel(topo)

In [ ]:
pde.model

Then, we define our optimization criteria and optimizer. More on this on Pytorch reference guide.

In [ ]:
# set optimization parameters
pde.setSolverParams(lr=0.01,epochs=100,batch_size=10)

### Solving the PDE

With everything in place, we can now solve the PDE. 

In [ ]:
pde.solve(each_epoch_print=10)

Depending on the problem configuration this will take more or less time. If the loss function does not goes to zero, something is wrong. 

### Evaluate the solution

Once the training is done, we can evaluate our solution.

In [ ]:
x = np.linspace(0,1,50)
t = np.linspace(0,1,4)
for _t in t:
    p0 = np.sin(2.*math.pi*(x-u*_t))
    p = pde.eval({'x': x, 't': np.array([_t])})
    plt.plot(x, p, 'k')
    plt.plot(x, p0)
    plt.show()

### Saving and loading the model

We can save the solution using standard pytorch functions. 

In [ ]:
import torch

PATH = 'MyPDE_solution.pt'

# save model
torch.save(pde.model.state_dict(), PATH)

Then we can load the model to continue training or just use it to get a solution. 

In [ ]:
pde.model.load_state_dict(torch.load(PATH))

When loading a model for evaluation, it is not required to set values for inputs/parameters. Nevertheless, to continue training all the previous steps are required.

### Author

Juan Sensio - juansensio03@gmail.com